1. TTA with WBF is not working with my YOLOv4.In this version I will use model trained on 1024x1024 resolution.

**Framework used for the training: Darknet**
**[https://github.com/AlexeyAB/darknet](http://)**

**License of the above framework:**
**[https://github.com/AlexeyAB/darknet/blob/master/LICENSE](http://)**

**++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

**Repository used for converting YOLOv4 weights into keras/TF2:**
**[https://github.com/david8862/keras-YOLOv3-model-set](http://)**

**License of the above repository:**
**[https://github.com/david8862/keras-YOLOv3-model-set/blob/master/LICENSE](http://)**

**++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

**Training Details:**
* I prepared the annotation files as per required for AlexeyAB/darknet's YOLOv4 from below kernel
**[https://www.kaggle.com/pabloberhauser/creating-label-files-for-use-in-yolov4](http://)**
 
* I followed following steps for the training:
**[https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects](http://)**

* After lots of trial with different parameters in configuration I achieved best mAP score in training-

without iou_thresh=0.05, mAP=0.931018

with iou_thresh=0.05, mAP=0.974559

* I performed the training in my local system with following configurations-
Windows 10, Core i7, GeForce GTX 1660 Ti, 16 GB RAM, CUDA 10.1, cuDNN 7.6.5

* The final training took around 36 hours in completion

**+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

**Preparing Submission File:**

I have made required changes and added my logic in following file '../input/yolo2keras/keras-YOLOv3-model-set/tools/evaluation/validate_yolo.py'

**+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

**Problem Faced:**

* Could not train the model with the higher resolution image in my system due to memory error.

* Could not able to use the original darknet command '.\darknet.exe detector test..' command to test the detection on new images. Since internet is not allowed and when I uploaded the compiled Darknet repo then I was getting permission denied error due to kaggl's input directory policy.

* Could not use OpenCV-Python DNN module with YOLOv4 since current version of OpenCV does not support it.

* Due to above issues, I converted the YOLOv4 weights into keras/TF2 and then make inference out of it. Making the submission file was not easy with this.

* Faced 'Submission csv not found' error many times while submitting the csv file.
**+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

**Experiments**

* with iou_threshold=0.4, score=0.6626
* with iou_threshold=0.05, score decreased to 0.6426
* with WBF code, score=0.6815
* with TTA and WBF, score=0.6135
* with improved TTA and WBF, score=0.6352


## YOLOv4 Training Result -

In [ ]:
# training chart showing mAP score and iteration details
from IPython.display import Image
Image("../input/wheat-yolov4-training-results/chart_wheat_608.png")

In [ ]:
# console output of the detection with default iou threshold
Image("../input/wheat-yolov4-training-results/detect_map_wheat.JPG")

In [ ]:
# console output of the detection with custom iou threshold
Image("../input/wheat-yolov4-training-results/detect_map_wheat_thres.JPG")

## Testing Result-

In [ ]:
Image("../input/wheat-yolov4-training-results/2fd875eaa.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/348a992bb.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/51b3e36ab.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/51f1be19e.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/53f253011.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/796707dd7.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/aac893a91.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/cb8d261a3.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/cc3532ff6.jpg")

In [ ]:
Image("../input/wheat-yolov4-training-results/f5a1f0358.jpg")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/working/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# import required libraries
import io
import time
import math
import cv2, colorsys
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

from functools import wraps, reduce

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.lite.python import interpreter as interpreter_wrapper
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Concatenate, MaxPooling2D, BatchNormalization, Activation, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2

print("TensorFlow version is: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version is: {}".format(tf.keras.__version__))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Below script will run on test images and then generate the submission file**

In [ ]:
!python ../input/yolo2keraswbf/keras-YOLOv3-model-set/tools/evaluation/validate_yolo.py --model_path=../input/yolo2keraswbf/keras-YOLOv3-model-set/weights/wheat_yolov4.h5 --image_file=../input/global-wheat-detection/test/ --anchors_path=../input/yolo2keraswbf/keras-YOLOv3-model-set/configs/yolo4_anchors.txt --classes_path=../input/yolo2kerastta/keras-YOLOv3-model-set/configs/wheat_classes.txt --model_image_size=1024x1024

In [ ]:
print("+++++++++++++++++++++++++++++++++ Completed +++++++++++++++++++++++++++++++++++++++++++")

**In this notebook I have taken care of licensing rule.**

**The repositories I am using in this notebook have licenses as required for this challenge.**

**My this notebook also demonstrates how can we use Darknet's YOLOv4 with Keras and TensorFlow.**

**We can also use OpenCV with YOLOv4 but current version of OpenCV does not support some new changes in YOLOv4.**

**We can run this notebook with or without GPU**